In [ ]:
import os, sys
import warnings

sys.path.insert(1, os.path.join(os.getcwd(), ".."))
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
import shallowsim as sb

In [ ]:
args = sb.KIMI_K2
c = sb.Config()
c.seq_len = 6000
c.decode_len = 200
c.kv_cache_rate = 0
c.tp_list = [1]
gpu_blackwell_decode = sb.get_gpu_info(
    "./device/gpu_info.csv", decoding_mode=True, print_console=True
)

# gpu_blackwell_decode['H20'].pcie_bw = 25 #change bw to 1.6Tbps..

In [ ]:
dfs = sb.decode_time_with_ep_list(
    args, gpu_blackwell_decode, c, fp8_combine=True, tps_limit=0, throughput_discount=0.75
)

# 推理性能天梯榜

In [ ]:
dfs_o = (
    dfs.groupby(["GPU", "BatchSize"], as_index=False)
    .apply(lambda t: t[t.Total == t.Total.max()])
    .sort_values(["Total"], ascending=False)
    .reset_index(drop=True)
)
dfs_o.style.bar(subset=["TPS", "Total"], color="#6495ED").applymap(
    sb.gpu_category_color,
    props=sb.gpu_category_idx(gpu_blackwell_decode),
    subset=["GPU"],
).applymap(sb.color_ratio, subset=["Profit_Ratio"]).applymap(
    sb.color_ratio, subset=["Profit_Ratio_O"]
).applymap(
    sb.color_positive_red, subset=["Delta"]
).background_gradient(
    subset=["Comm_Impact"], cmap=sb.cm
).format(
    precision=3
)

# 查看单个GPU的性能

In [ ]:
sb.df_filter(dfs, "H800", device_num=32).style.bar(
    subset=["TPS", "Total"], color="#6495ED"
).applymap(sb.color_positive_red, subset=["Delta"]).applymap(
    sb.color_ratio, subset=["Profit_Ratio"]
).applymap(
    sb.color_ratio, subset=["Profit_Ratio_O"]
).background_gradient(
    subset=["Comm_Impact"], cmap=sb.cm
).format(
    precision=3
)

In [ ]:
sb.df_filter(dfs, "H200", device_num=32).style.bar(
    subset=["TPS", "Total"], color="#6495ED"
).applymap(sb.color_positive_red, subset=["Delta"]).applymap(
    sb.color_ratio, subset=["Profit_Ratio"]
).applymap(
    sb.color_ratio, subset=["Profit_Ratio_O"]
).background_gradient(
    subset=["Comm_Impact"], cmap=sb.cm
).format(
    precision=3
)

In [ ]:
cols = [
    "GPU",
    "TP",
    "EP",
    "BatchSize",
    "Delta",
    "TPS",
    "TPS_O",
    "Total",
    "Total_O",
    "Cost",
    "Revenue",
    "Revenue_O",
    "Profit_Ratio",
    "Profit_Ratio_O",
    "Comm_Impact",
]
sb.df_filter(dfs[cols], "H20-3E", device_num=16).style.bar(
    subset=["TPS", "Total"], color="#6495ED"
).applymap(sb.color_positive_red, subset=["Delta"]).applymap(
    sb.color_ratio, subset=["Profit_Ratio"]
).applymap(
    sb.color_ratio, subset=["Profit_Ratio_O"]
).background_gradient(
    subset=["Comm_Impact"], cmap=sb.cm
).format(
    precision=3
)
